## Projeto de Análise de dados - Estatística - Profa. Maria Luísa - ADS III

### Alunos:
- João Victor Carrijo
- Danilo de Andrade
- Eloísa Rodrigues

### Sobre o projeto
Neste trabalho analisaremos uma base de dados envolvendo os dados de vendas de vídeo games

Créditos: https://www.kaggle.com/datasets/gregorut/videogamesales

### Importação de Bibliotecas

In [87]:
import pandas as pd # leitura de dados
import numpy as np # cálculos
import plotly.express as plt # visualização de dados

## Pré-Processamento

### Lendo dados e fazendo ajustes iniciais

In [88]:
# Lendo os dados
df_games = pd.read_csv('games-raw.csv')
df_consoles = pd.read_csv('consoles-raw.csv')

In [89]:
# Renomeando colunas
df_games = df_games.rename(
    columns={
        'Name':'Titulo',
        'Platform':'Plataforma',
        'Year':'Ano_lancamento',
        'Genre':'Genero',
        'Publisher':'Distribuidora',
        'NA_Sales':'Vendas_EUA',
        'EU_Sales':'Vendas_Europa',
        'JP_Sales':'Vendas_JP',
        'Other_Sales':'Outras_Vendas',
        'Global_Sales':'Vendas_Totais'
    }
)

In [ ]:
df_consoles = df_consoles.rename(
    columns={
        'Console Name':'Nome do Console',
        'Type':'Tipo',
        'Company':'Fabricante',
        'Released Year':'Ano de Lançamento',
        'Discontinuation Year':'Ano de Descontinuacao',
        'Units sold (million)':'Unidades vendidas (Milhões)'
    }
)
df_consoles = df_consoles.drop(columns='Remarks')
df_consoles

### Identificando e tratando dados nulos

In [ ]:
df_games.info()

# Valores Nulos:

# Ano_lancamento -> 271
# Distribuidora -> 58
df_games['Distribuidora'][df_games['Distribuidora'].isna()]

Neste dataset, temos alguns dados não contabilizados, ou então não divulgados a respeito dos jogos.
A boa prática diz que dados inconsistentes deste tipo devem ser preenchidos com a média da coluna, para que estes não prejudiquem a nossa análise. 

In [92]:
# Vamos fazer a média do ano de lançamento
media_ano = np.round(df_games['Ano_lancamento'].mean())
media_ano
df_games['Ano_lancamento'] = df_games['Ano_lancamento'].fillna(media_ano)

In [ ]:
df_games.info()

## Tratamento Geral

### Traduzindo Atributos

In [ ]:
# Gêneros
def traduz_genero(x):
    if x == 'Sports':
        return 'Esportes'
    if x == 'Action':
        return 'Ação'
    if x == 'Simulation':
        return 'Simulação'
    if x == 'Racing':
        return 'Corrida'
    if x == 'Platform':
        return 'Plataforma'
    if x == 'Role-Playing':
        return 'RPG'
    if x == 'Shooter':
        return 'Tiro'
    if x == 'Fighting':
        return 'Luta'
    if x == 'Adventure':
        return 'Aventura'
    if x == 'Strategy':
        return 'Estratégia'
    return x
    

df_games['Genero'] = df_games['Genero'].apply(traduz_genero)
df_games['Genero']

In [ ]:
# Tipos de Console
def traduz_tipo(x):
    if x == 'Home':
        return 'De mesa'
    if x == 'Handheld':
        return 'Portátil'
    if x == 'Hybrid':
        return 'Híbrido'

df_consoles['Tipo'] = df_consoles['Tipo'].apply(traduz_tipo)
df_consoles['Tipo']

### Agrupando vendas do mesmo jogo em múltiplas plataformas

In [ ]:
all_platforms = df_games.groupby('Titulo').agg({
    'Ano_lancamento': lambda x: x.mode()[0], # Pega o dado de maior frequência (moda)
    'Genero': lambda x: x.mode(),
    'Vendas_Totais':lambda x : np.round(sum(x),2), # Soma tudo e arredonda
    'Vendas_JP': lambda x : np.round(sum(x),2),
    'Vendas_EUA':lambda x : np.round(sum(x),2),
    'Vendas_Europa':lambda x : np.round(sum(x),2),
    'Outras_Vendas':lambda x : np.round(sum(x),2)
}).reset_index()
all_platforms

In [ ]:
all_platforms.sort_values('Vendas_Totais', ascending=False)

In [ ]:
all_platforms.fillna(0)

### Adicionando coluna "geração"

A indústria dos vídeogames é dividida por gerações de consoles, a transição entre gerações geralmente ocorre com o lançamento de um novo console com poder de processamento superior a tudo aquilo que já existe no mercado, em especial no que se refere ao poder de processamento.

Portanto, podemos realizar um agrupamento baseado em gerações, para que possamos ter uma visão ampla de nossos dados.

In [ ]:
# Observação: para uma manipulação eficiente, devemos lidar com uma variável do tipo DataFrame, e não do tipo Series.
df_consoles = pd.DataFrame(df_consoles)
df_consoles

def get_gen(row):
    if row['Ano de Lançamento']<1981:
        return 2 # Atari 
    elif row['Ano de Lançamento']<1988:
        return 3 # NES e Master System
    elif row['Ano de Lançamento']<1994:
        return 4 # SNES e Genesis
    elif row['Ano de Lançamento']<1998:
        return 5 # PS1, Saturn, N64
    elif row['Ano de Lançamento']<2005: 
        return 6 # PS2, DreamCast, XB
    elif row['Ano de Lançamento']<2012:   
        return 7 # X360, PS3
    else: 
        return 8 # XONE, PS4

df_consoles['Geração'] = df_consoles.apply(get_gen, axis=1)
df_consoles


### Adicionando Coluna de "Duração no mercado"

In [ ]:
df_consoles['Duracao no Mercado'] = df_consoles['Ano de Descontinuacao'] - df_consoles['Ano de Lançamento']
# Tratando Gerações atuais:
df_consoles['Duracao no Mercado'] = df_consoles['Duracao no Mercado'].apply(lambda x: x if x>0 else x+2024) 
df_consoles

### Corrigindo Coluna Ano do Lançamento

In [ ]:
df_games['Ano_lancamento'] = df_games['Ano_lancamento'].apply(lambda x : f'01-01-{round(x)}')
df_games

## Visualização Básica

In [ ]:
# Visualização de dados descritivos das colunas numéricas
df_games.describe()

In [111]:
print(np.round(df_games['Vendas_Totais'].mean(),2), df_games['Vendas_Totais'].mode(), df_games['Vendas_Totais'].median())

0.54 0    0.02
Name: Vendas_Totais, dtype: float64 0.17


In [ ]:
# Visualizando jogo com maior quantidade de vendas no total
mais_vendas = df_games['Vendas_Totais'] >= 82.74
df_games[mais_vendas]

In [ ]:
# Visualizando jogo(s) com menor venda nos EUA
pior_avaliacao = df_games['Vendas_EUA'] <= 0.0
df_games[pior_avaliacao]

In [ ]:
# Retornando valores únicos
np.unique(df_games['Plataforma']), np.unique(df_consoles['ID'])


In [ ]:
# Buscando...
jogo = df_games['Titulo'] == 'Grand Theft Auto: San Andreas'
df_games[jogo]

Neste dataset, temos alguns dados não contabilizados, ou então não divulgados a respeito dos jogos.
A boa prática diz que dados inconsistentes deste tipo devem ser preenchidos com a média da coluna, para que estes não prejudiquem a nossa análise. 

### Visualizando a dispersão

Agora que resolvemos alguns problemas do dataset, vamos visualizar alguns gráficos para conferir se ainda restam outliers.

In [ ]:
pd.DataFrame.iteritems = pd.DataFrame.items # Na última atualização do pandas, o método "iteritems", que ainda é utilizado
# pelo plotly trocou de nome, por tanto essa atrivbuição resolverá um bug na comunicação da duas biblioteca. 

# Vendas totais x Críticas x Vendas EUA X Vendas Japão 
plt.scatter_matrix(df_games, dimensions=['Vendas_Totais','Vendas_JP','Vendas_Europa', 'Vendas_EUA'])

### Insights Prévios

In [ ]:
# Gênero por console
plt.treemap(df_games, path=['Genero', 'Plataforma'])

In [ ]:
# Gênero por vendas totais
plt.pie(df_games,names='Genero',values='Vendas_Totais')

In [ ]:
# Score por Console
plt.bar(df_games,x='Plataforma',y='Vendas_Totais')


## Exportando dados para um arquivo CSV

In [118]:
df_games.to_csv('games-Final.csv', index=False)
df_consoles.to_csv('consoles-Final.csv', index=False)
all_platforms.to_csv('all-platforms.csv', index=False)